In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/AI/Final/


/content/drive/MyDrive/Colab Notebooks/AI/Final


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/AI/Final/

/content/drive/.shortcut-targets-by-id/1u7XjJzXhLq6aFZqOV6XBp3ZHv91O4fv5/AI/Final


In [33]:
import pandas as pd 
import numpy as np

from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.pipeline import Pipeline
from random import randint

In [34]:
df = pd.read_csv("disgust.csv")

In [35]:
df.head()

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,...,p_30,p_31,p_32,p_33,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,emotion
0,1,0,0,0.98,1,0.215443,-0.064069,-0.974412,-0.133780,-0.055220,-0.989471,0.042,-0.061,12.0,12.6,14.1,15.6,16.3,15.8,14.2,12.7,9.9,10.9,12.2,13.9,15.5,16.8,17.7,16.7,15.3,13.8,12.3,10.9,13.5,14.2,14.8,15.1,14.8,14.1,13.5,...,0.006,0.000,0.006,0.005,0.17,0.85,0.84,0.0,0.95,1.88,0.00,0.58,0.46,0.00,1.08,1.31,0.17,0.0,0.79,0.44,0.00,1,1,1,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1
1,2,0,0,0.98,1,0.212373,-0.009683,-0.977141,-0.143409,-0.001292,-0.989663,0.035,-0.006,12.5,13.1,14.5,16.0,16.6,16.1,14.6,13.1,10.5,11.4,12.7,14.3,15.9,17.1,18.0,17.0,15.6,14.1,12.7,11.4,13.9,14.5,15.2,15.5,15.2,14.5,13.9,...,0.006,0.001,-0.001,0.004,0.23,0.83,0.90,0.0,0.79,2.17,0.03,0.43,0.15,0.00,1.16,1.41,0.13,0.0,0.83,0.43,0.00,0,1,1,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1
2,3,0,0,0.98,1,0.229896,0.067599,-0.970865,-0.133035,0.076322,-0.988168,0.049,0.073,12.1,12.7,14.2,15.7,16.3,15.8,14.2,12.7,10.1,11.0,12.3,14.0,15.6,16.8,17.7,16.7,15.3,13.8,12.3,11.0,13.6,14.2,14.9,15.1,14.9,14.2,13.6,...,0.005,-0.002,-0.002,0.004,0.51,0.62,1.05,0.0,0.55,2.54,0.21,0.44,0.00,0.03,0.80,1.20,0.27,0.0,0.78,0.75,0.08,0,1,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,1
3,4,0,0,0.98,1,0.207316,-0.002886,-0.978270,-0.132507,0.006576,-0.991160,0.038,0.002,13.3,13.8,15.2,16.5,17.1,16.7,15.2,13.9,11.5,12.3,13.5,15.0,16.4,17.6,18.4,17.5,16.2,14.8,13.5,12.3,14.6,15.2,15.8,16.0,15.8,15.2,14.6,...,-0.001,0.001,0.001,-0.001,0.78,0.48,1.16,0.0,0.37,2.80,0.40,0.63,0.00,0.03,0.44,1.08,0.40,0.0,0.89,1.33,0.14,1,0,1,1,1,1,0,1,0,0,1,1,0,0,0,0,0,0,1
4,5,0,0,0.98,1,0.198652,-0.001479,-0.980069,-0.138150,0.008748,-0.990373,0.031,0.004,13.7,14.2,15.6,16.9,17.5,17.0,15.6,14.3,11.9,12.7,13.9,15.4,16.8,17.9,18.7,17.8,16.6,15.2,13.9,12.8,15.0,15.6,16.2,16.4,16.2,15.6,15.0,...,-0.002,0.002,0.002,0.000,0.95,0.11,1.24,0.0,0.24,3.08,0.56,0.79,0.00,0.03,0.04,0.99,0.57,0.0,1.05,1.91,0.25,0,0,1,1,1,1,0,1,0,0,1,1,0,1,0,0,0,0,1


In [36]:
data = df.iloc[:,-36:]
data = data.sample(frac=1, random_state = 42).reset_index(drop = True)
data

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,emotion
0,0.01,0.13,1.08,0.00,0.22,2.35,0.00,0.00,0.00,0.61,0.00,0.50,0.34,0.01,0.55,0.25,0.36,0,0,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,1
1,0.85,0.25,0.49,0.00,0.00,0.49,0.00,0.00,0.00,0.15,1.23,1.91,1.27,0.18,0.27,1.65,0.02,0,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0
2,0.00,0.00,0.81,0.00,0.00,0.24,0.00,0.00,0.00,0.00,0.86,2.20,0.00,0.00,0.28,0.21,0.00,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0
3,0.68,0.74,0.18,0.14,0.17,0.22,0.00,0.21,0.82,0.53,0.13,0.58,0.00,0.00,0.67,0.89,0.03,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0
4,0.10,0.26,0.28,0.10,0.21,1.30,0.00,0.46,0.11,0.21,0.50,0.21,0.21,0.00,0.57,0.10,0.17,0,0,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,0.10,0.11,1.15,0.00,0.53,2.20,0.26,0.18,0.05,0.23,0.13,0.42,0.19,0.02,0.00,0.18,0.00,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1
346,0.47,0.02,0.89,0.00,0.01,1.46,0.19,0.00,0.00,0.00,0.00,1.38,0.00,0.29,2.30,0.44,0.00,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
347,0.76,0.52,0.00,0.00,0.32,0.48,0.42,0.14,0.69,0.32,0.00,0.52,0.01,0.01,0.57,1.38,0.00,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
348,0.95,0.03,0.91,0.21,0.00,0.05,0.04,0.00,0.09,0.30,1.23,1.92,0.11,0.20,0.34,0.00,0.05,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0


In [37]:
aur = data.iloc[:,:35]
#auc = data.iloc[:,17:35]
y = data.iloc[:,-1]
m = y.shape[0]
col = aur.shape[1]


#GA Algorithm

In [38]:
def GA(ng = 5, psize = 20, method = 'uniform', classf = 'dtc'):
  global pop, fit
  pop = population(psize)
  fit = []
  for i in range(psize):
    fit.append(fitness(pop[i], classf))

  for i in range(ng):
    if len(pop)==1:
      break
    Xover(psize, method, classf)
  return fit[0]

##Population


In [39]:
def individual(i):
  np.random.seed(42+i)
  return (np.random.choice([1,0], size=col)).tolist()

def population(count=20):
  return [individual(x) for x in range(count)]

#Fitness

In [40]:
def fitness(arr, i=0, classf = 'dtc'):
  ind = np.array(arr).reshape((1,col))
  mul = pd.DataFrame( np.ones((m,1)) @ ind )
  mod = pd.DataFrame(aur.values*mul.values,columns=aur.columns,index=aur.index)

  np.random.seed(42)
  xtrain, xtest, ytrain, ytest = train_test_split(mod, y, random_state = 42, stratify = y)

  global pipe, params, clf
  if classf == 'dtc':
    pipe = Pipeline([('clf',DTC())])
    params = {'clf__criterion':['gini', 'entropy']}
  elif classf == 'knn':
    pipe = Pipeline([('clf',KNC())])
    params = {'clf__n_neighbors':[5,7,10,15,20]}
  elif classf == 'rfc':
    pipe = Pipeline([('clf',RFC())])
    params = {'clf__n_estimators':[10,20,30,50]}
  
  gs = GridSearchCV(pipe, param_grid = params, cv = 5)
  gs.fit(xtrain, ytrain)
  

  if classf == 'dtc':
    crit = gs.best_params_['clf__criterion']
    clf = DTC(criterion=crit)
  elif classf == 'knn':
    crit = gs.best_params_['clf__n_neighbors']
    clf = KNC(n_neighbors = crit)
  elif classf == 'rfc':
    crit = gs.best_params_['clf__n_estimators']
    clf = RFC(n_estimators = crit)

  clf.fit(xtrain, ytrain)
  acc = clf.score(xtest, ytest)

  if i==1:
  #########################################################################################
    from sklearn.externals.six import StringIO  
    from IPython.display import Image  
    from sklearn.tree import export_graphviz
    import pydotplus
    feature_cols = aur.columns.tolist()
    dot_data = StringIO()
    export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['not disgust','disgust'])
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png('knn.png')
  #########################################################################################
  
  return (acc/sum(arr))

#Crossover

In [41]:
def Xover(psize, method = 'uniform', classf = 'dtc'):
  global pop, fit
  
  sorted_df = pd.DataFrame({'fit':fit, 'pop':pop})
  sorted_df = sorted_df.sort_values(by = 'fit', ascending = False).reset_index(drop = True)
  fit, pop = sorted_df['fit'].tolist(), sorted_df['pop'].tolist()

  pop2 = []
  j = len(pop)
  
  if method == 'uniform':
    for i in range(0,j,2):
      child1, child2 = [], []
      if i==j-1:
        break
      for k in range(col):
        x=np.random.randint(2)
        if(x==0):
          child1.append(pop[i][k])
          child2.append(pop[i+1][k])
        else:
          child1.append(pop[i+1][k])
          child2.append(pop[i][k])
      pop2.append(child1)
      pop2.append(child2)

  if method == 'one point':
    for i in range(0,j,2):
      x = np.random.randint(col)
      if i==j-1:
        break
      pop2.append(pop[i][:x] + pop[i+1][x:])
      pop2.append(pop[i+1][:x] + pop[i][x:])
  
  fit2 = []

  for i in range(len(pop2)):
    fit2.append(fitness(pop2[i], classf))
  pop.extend(pop2)
  fit.extend(fit2)
  
  sorted_df = pd.DataFrame({'fit':fit, 'pop':pop})
  
  sorted_df.drop_duplicates(subset='fit', inplace=True)
  
  sorted_df = sorted_df.sort_values(by = 'fit', ascending = False).reset_index(drop = True)
  fit, pop = sorted_df['fit'].tolist(), sorted_df['pop'].tolist()
  lslen = len(pop)
  if lslen>psize:
    fit, pop = fit[:psize], pop[:psize]
  

  

#Feature subset selection

In [42]:
GA(psize=21, method = 'uniform')
fit

[0.12175324675324675,
 0.09848484848484848,
 0.09343434343434344,
 0.09090909090909091,
 0.08863636363636364,
 0.08838383838383838,
 0.0875,
 0.08522727272727273,
 0.08471074380165289,
 0.08409090909090909,
 0.08295454545454546,
 0.08264462809917356,
 0.08161157024793388,
 0.08057851239669421,
 0.07747933884297521,
 0.07575757575757576,
 0.07481060606060606,
 0.0743801652892562,
 0.07386363636363637,
 0.07334710743801652,
 0.07291666666666667]

#Highest Individual Fitness

In [48]:
fitness(pop[0],i=1)

0.12175324675324675

In [44]:
fcol = []
def getcol():
  global fcol
  fitind = np.array(pop[0])
  fitind = (fitind==1)
  fcol = np.arange(len(pop[0]))
  fcol = fcol[fitind].tolist()
getcol()

##Action Units which are necessary:

In [45]:
print(list(aur.iloc[:,fcol].columns))

[' AU07_r', ' AU23_r', ' AU26_r', ' AU09_c', ' AU12_c', ' AU17_c', ' AU23_c']


#Classification

##Getting column number list to get from the dataset

In [16]:
fcol = []
def getcol():
  global fcol
  fitind = np.array(pop[0])
  fitind = (fitind==1)
  fcol = np.arange(len(pop[0]))
  fcol = fcol[fitind].tolist()
getcol()

###Data Pre-processing for classification

In [ ]:
cldata = pd.read_csv('disgust_team.csv')
cldata = cldata.iloc[:,-36:]
cldata = cldata.sample(frac=1, random_state = 42).reset_index(drop = True)
cldata

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,AU15_r,AU17_r,AU20_r,AU23_r,AU25_r,AU26_r,AU45_r,AU01_c,AU02_c,AU04_c,AU05_c,AU06_c,AU07_c,AU09_c,AU10_c,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,emotion
0,1.66,1.01,0.04,1.27,0.00,0.00,0.15,0.00,0.01,0.00,0.00,0.34,0.00,0.00,0.61,0.94,0.45,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2.74,0.34,0.09,0.00,0.86,0.48,0.61,2.80,1.35,1.40,1.59,2.78,0.00,0.00,0.00,1.82,0.00,1,1,0,1,0,1,0,1,0,1,1,1,0,0,0,0,0,0,1
2,1.35,0.22,0.51,0.53,0.39,0.22,0.12,0.71,0.56,1.12,0.97,1.34,0.25,0.00,1.01,0.73,0.73,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0.00,0.00,0.49,0.00,1.39,1.17,0.00,1.09,0.97,1.32,0.64,1.79,0.27,2.56,0.71,0.76,0.00,0,0,1,0,1,1,0,1,1,1,0,0,0,1,1,1,0,0,1
4,0.00,0.00,0.74,0.00,0.17,0.16,0.91,0.20,0.12,0.20,0.55,1.11,0.23,0.54,1.85,0.53,0.33,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2.35,1.01,0.28,1.27,0.00,0.00,0.15,0.00,0.05,0.20,0.28,0.58,0.00,0.00,0.35,0.70,0.45,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
67,1.01,0.00,0.63,0.00,0.37,0.00,0.00,0.00,0.65,1.05,0.27,0.90,0.04,0.06,0.82,1.21,0.09,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
68,0.50,0.18,0.85,0.05,0.21,0.00,0.00,0.03,0.46,0.15,0.16,0.90,1.48,0.00,1.40,1.32,0.00,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0
69,0.00,0.00,0.71,0.00,0.00,0.04,0.04,0.65,0.38,0.83,1.06,1.89,0.00,0.01,0.07,0.80,0.00,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0


In [ ]:
clx = cldata.iloc[:,:35]
cly = cldata.iloc[:,-1]
clm = cly.shape[0]
clcol = clx.shape[1]

###Classifiers

In [ ]:
def classify(arr=pop[0], i = 0, classf = 'dtc'):
  '''
  ind = np.array(arr).reshape((1,clcol))
  mul = pd.DataFrame( np.ones((clm,1)) @ ind )
  mod = pd.DataFrame(clx.values*mul.values,columns=clx.columns,index=clx.index)

  np.random.seed(42)
  xtrain, xtest, ytrain, ytest = train_test_split(mod, cly, random_state = 42, stratify = cly)
  '''
  #np.random.seed(42)
  #xtrain, xtest, ytrain, ytest = train_test_split(clx.iloc[:,fcol], cly, random_state=42, stratify = cly)

  xtrain, xtest, ytrain, ytest = aur.iloc[:,fcol], clx.iloc[:,fcol], y, cly
  global pipe, params, clf
  if classf == 'dtc':
    pipe = Pipeline([('clf',DTC())])
    params = {'clf__criterion':['gini', 'entropy']}
  elif classf == 'knn':
    pipe = Pipeline([('clf',KNC())])
    params = {'clf__n_neighbors':[5,7,10,15,20]}
  elif classf == 'rfc':
    pipe = Pipeline([('clf',RFC())])
    params = {'clf__n_estimators':[10,20,30,50]}
  
  gs = GridSearchCV(pipe, param_grid = params, cv = 5)
  gs.fit(xtrain, ytrain)
  

  if classf == 'dtc':
    crit = gs.best_params_['clf__criterion']
    clf = DTC(criterion=crit)
  elif classf == 'knn':
    crit = gs.best_params_['clf__n_neighbors']
    clf = KNC(n_neighbors = crit)
  elif classf == 'rfc':
    crit = gs.best_params_['clf__n_estimators']
    clf = RFC(n_estimators = crit)

  clf.fit(xtrain, ytrain)
  acc = clf.score(xtest, ytest)

  if i==1:
  #########################################################################################
    from sklearn.externals.six import StringIO  
    from IPython.display import Image  
    from sklearn.tree import export_graphviz
    import pydotplus
    feature_cols = xtrain.columns.tolist()
    dot_data = StringIO()
    export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['not disgust','disgust'])
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png('dtcteam.png')
  #########################################################################################
  
  return acc

In [ ]:
classify()

0.8028169014084507

#End of the code

#Appending Results to CSVs

###Final Fitness with different:
>Number of Generations & Population Size and Crossover Methods


In [ ]:
ngls = [5,5,5,5,10,10,10,20,20,20,40,40]
psizels = [5,10,20,40,5,20,40,10,20,40,20,40]
methls = ['dtc', 'knn', 'rfc']

In [ ]:
unifls = []
unifcls = []
for i in methls:
  templs = []
  tcls = []
  for j in range(len(ngls)):
    templs.append(GA(ng=ngls[j], classf = i, method='one point', psize=psizels[j]))
    getcol()
    tcls.append(classify(classf=i))
  unifls.append(templs)
  unifcls.append(tcls)

In [ ]:
pd.DataFrame(unifls).to_csv('unifdf.csv')
pd.DataFrame(unifcls).to_csv('unifcls.csv')

#Testing Codes

In [ ]:
jhangir = np.array([1,0,1,0,0,1,1,1,0,0,1])
jhangir.shape

(11,)

In [ ]:
fjh = (jhangir==1)

In [ ]:
fjh
fjh1 = np.arange(fjh.shape[0])
fjh1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
fjh1 = fjh1[fjh].tolist()

In [ ]:
fjh1

[0, 2, 5, 6, 7, 10]